## DL3
Follow this notebook only if you're new to DeepLearing and Transfer learning. This is a extension of the Startet kit given [here](https://github.com/shubham3121/DL-3/blob/master/DL%233_EDA.ipynb). I'll try to keep it simple. Please ignore the typos :)

### Why to use Mobilenet architecture?
You might have seen multiple tutorials on the VGG16 based transfer learning but here I'm going to use Mobilenet because of the following reasons 
<ul>
    <li> No. of parameters to train in Mobilenet is quite less in compare to the VGG16
    <li> Having fewer parameters will make your training time less and you'll be able to do more experiment and your chances of wining becames higher.
    <li> On top of above reasons Mobile net has similar performance on the ImageNet dataset as VGG16
</ul>

Having said that let move on to the imorting important libs

In [12]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import MobileNet
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model

from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
import gc
import cv2 as cv

In [2]:
import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/keras-team/keras/issues/2280#issuecomment-306959926

import os
os.environ['PYTHONHASHSEED'] = '0'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of
# non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)


I'm going to use 128x128 images. You can change that if you wish.

My folder structure is as follow

<ul>
    <li> DL3</li>
        <ul>
            <li> starter_kit</li>
                <ul>
                    <li> this_notebook</li>
                </ul>
            <li> data</li>
                <ul>
                    <li> train_img</li>
                    <li> test_img</li>
                </ul>
    </ul>
</ul>

In [5]:
img_width, img_height = (128, 128)

train_data_dir = '../data/train_img/'
test_data_dir = '../data/test_img/'
epochs = 10
batch_size = 128

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [6]:
Mobile_model = MobileNet(include_top=False, input_shape=input_shape)

17227776/17225924 [==============================] - 41s 2us/step


In [20]:
def get_model():
# add a global spatial average pooling layer
    x = Mobile_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(85, activation='sigmoid')(x)
    model = Model(inputs=Mobile_model.input, outputs=predictions)
    
    return model

In [32]:
model = get_model()

We'll start with training the head(last layer) only as that layer is initialized randomaly and we don't want to affect the other layers weights as while backpropogation. 

In [33]:
#train only last layer
for layer in model.layers[:-1]:
    layer.trainable = False

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 130, 130, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 66, 66, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
__________

In [23]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(width_shift_range=0.2, height_shift_range=0.2,
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True, rotation_range = 20)

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

In [43]:
train = pd.read_csv('../train.csv', index_col=0)
test = pd.read_csv('../test.csv')
attributes = pd.read_csv('../attributes.txt', delimiter='\t', header=None, index_col=0)
classes = pd.read_csv('../classes.txt', delimiter='\t', header=None, index_col=0)

In [13]:
def get_imgs(src, df, labels = False):
    if labels == False:
        imgs = []    
        files = df['Image_name'].values
        for file in tqdm(files):
            im = cv.imread(os.path.join(src, file))
            im = cv.resize(im, (img_width, img_height))
            imgs.append(im)
        return np.array(imgs)
    else:
        imgs = []
        labels = []
        files = os.listdir(src)
        for file in tqdm(files):
            im = cv.imread(os.path.join(src, file))
            im = cv.resize(im, (img_width, img_height))
            imgs.append(im)
            labels.append(df.loc[file].values)
        return np.array(imgs), np.array(labels)

In [14]:
train_imgs, train_labels = get_imgs(train_data_dir, train, True)

100%|██████████| 12600/12600 [03:09<00:00, 66.47it/s]


In [15]:
#train val split
X_tra, X_val, y_tra, y_val = train_test_split(train_imgs, train_labels, test_size = 3000, random_state = 222)
gc.collect()

92079

In [27]:
train_datagen.fit(X_tra)
val_datagen.fit(X_val)

In [17]:
def fmeasure(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

We're going to train our model with SGD and very low learning rate

In [34]:
early_stp = EarlyStopping(patience=3)
model_ckpt = ModelCheckpoint('mobilenet_1_layer.h5', save_weights_only=True)

opt = optimizers.SGD(lr=0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(opt, loss = 'binary_crossentropy', metrics=['accuracy', fmeasure])

In [35]:
model.fit_generator(train_datagen.flow(X_tra, y_tra, batch_size=batch_size),                     
                    steps_per_epoch=len(X_tra) / batch_size, epochs=5,
                    validation_data=val_datagen.flow(X_val, y_val, batch_size=batch_size), 
                    validation_steps = len(X_val)/batch_size, callbacks=[early_stp, model_ckpt], workers = 10, max_queue_size=20)

Epoch 1/5
75/75 [==============================] - 16s 216ms/step - loss: 0.7681 - acc: 0.5849 - fmeasure: 0.4916 - val_loss: 0.6558 - val_acc: 0.6546 - val_fmeasure: 0.5498
Epoch 2/5
75/75 [==============================] - 13s 177ms/step - loss: 0.6039 - acc: 0.6926 - fmeasure: 0.5845 - val_loss: 0.5644 - val_acc: 0.7195 - val_fmeasure: 0.6152
Epoch 3/5
75/75 [==============================] - 14s 184ms/step - loss: 0.5437 - acc: 0.7348 - fmeasure: 0.6297 - val_loss: 0.5185 - val_acc: 0.7492 - val_fmeasure: 0.6490
Epoch 4/5
75/75 [==============================] - 13s 177ms/step - loss: 0.5092 - acc: 0.7561 - fmeasure: 0.6552 - val_loss: 0.4874 - val_acc: 0.7679 - val_fmeasure: 0.6730
Epoch 5/5
75/75 [==============================] - 13s 180ms/step - loss: 0.4839 - acc: 0.7705 - fmeasure: 0.6741 - val_loss: 0.4630 - val_acc: 0.7822 - val_fmeasure: 0.6922


In [37]:
model = get_model()

#train only last 10 layer
for layer in model.layers:
    layer.trainable = True

opt = optimizers.SGD(lr=0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(opt, loss = 'binary_crossentropy', metrics=['accuracy', fmeasure])
    
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 130, 130, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 66, 66, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
__________

In [38]:
early_stp = EarlyStopping(patience=3)
model_ckpt = ModelCheckpoint('mobilenet_all_layers.h5', save_weights_only=True)
model.load_weights('mobilenet_1_layer.h5')


In [40]:
model.fit_generator(train_datagen.flow(X_tra, y_tra, batch_size=batch_size),                     
                    steps_per_epoch=len(X_tra) / batch_size, epochs=10,
                    validation_data=val_datagen.flow(X_val, y_val, batch_size=batch_size), 
                    validation_steps = len(X_val)/batch_size, callbacks=[early_stp, model_ckpt], workers = 10, max_queue_size=20)

Epoch 1/10
75/75 [==============================] - 32s 422ms/step - loss: 0.4556 - acc: 0.7866 - fmeasure: 0.6967 - val_loss: 0.4296 - val_acc: 0.8015 - val_fmeasure: 0.7172
Epoch 2/10
75/75 [==============================] - 30s 396ms/step - loss: 0.4270 - acc: 0.8026 - fmeasure: 0.7193 - val_loss: 0.4070 - val_acc: 0.8141 - val_fmeasure: 0.7341
Epoch 3/10
75/75 [==============================] - 30s 396ms/step - loss: 0.4032 - acc: 0.8163 - fmeasure: 0.7390 - val_loss: 0.3891 - val_acc: 0.8245 - val_fmeasure: 0.7489
Epoch 4/10
75/75 [==============================] - 30s 397ms/step - loss: 0.3846 - acc: 0.8269 - fmeasure: 0.7546 - val_loss: 0.3741 - val_acc: 0.8330 - val_fmeasure: 0.7613
Epoch 5/10
75/75 [==============================] - 30s 396ms/step - loss: 0.3674 - acc: 0.8365 - fmeasure: 0.7683 - val_loss: 0.3614 - val_acc: 0.8406 - val_fmeasure: 0.7722
Epoch 6/10
75/75 [==============================] - 30s 396ms/step - loss: 0.3539 - acc: 0.8446 - fmeasure: 0.7802 - val_loss

In [44]:
test_imgs = get_imgs(test_data_dir, test)

100%|██████████| 5400/5400 [01:24<00:00, 65.45it/s]


In [45]:
test_datagen.fit(test_imgs)

In [47]:
pred = model.predict_generator(test_datagen.flow(test_imgs, batch_size=512, shuffle=False), verbose=1, workers=8)

11/11 [==============================] - 5s 449ms/step


In [48]:
sub = pd.read_csv('../sample_submission.csv')
sub.iloc[:, 1:] = pred.round().astype(int)
sub.head()

,Image_name,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
0,Image-1.jpg,0,0,0,1,0,1,0,0,1,...,0,0,0,1,1,0,0,1,0,0
1,Image-2.jpg,0,1,0,1,1,1,1,0,0,...,0,1,0,1,0,0,0,0,1,1
2,Image-3.jpg,0,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
3,Image-4.jpg,0,0,0,0,0,1,0,0,1,...,1,1,1,1,0,0,0,1,0,0
4,Image-5.jpg,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [49]:
sub.to_csv('submission.csv', index=False)

In [50]:
sub.shape

(5400, 86)

## Final Thoughts

This submission should get you around $\approx$0.80 on the LB. and if you've noticed that our last epochs val_fmeasure is the same so it means that our val set is represantion of the test set and you can train for many epochs with EarlyStopping without worring about overfitting on val or train set.

### How to improve from here?
You can change many things which will let you get higher LB score. Following is a small list
<ul>
    <li> Change the Image size to bigger number </li>
    <li> Increse the number of epoch in the fully trainable network($2^{nd}$ training) </li>
    <li> Use diffrent architechure. you'll get more info on that [here](keras.io/applications/)</li>
    <li> If nothing works ensemble is your best friend </li>
</ul>

I'll try to keep improving this notebook. Feel free to contribuite.

Thanks
